# This notebook is code to accompany article Speed up your Pandas Processing with Swifter

Article: https://towardsdatascience.com/speed-up-your-pandas-processing-with-swifter-6aa314600a13?sk=e97035b67d0b3cd89770fe63b9321d91

Dataset: https://www.kaggle.com/unanimad/dataisbeautiful

In [1]:
import pandas as pd
import swifter

df = pd.read_csv('r_dataisbeautiful_posts.csv')

C:\Users\MSI\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (5,7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
import pandas as pd
import swifter

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183891 entries, 0 to 183890
Data columns (total 13 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   id                     183891 non-null  object 
 1   title                  183890 non-null  object 
 2   score                  183891 non-null  int64  
 3   author                 183891 non-null  object 
 4   author_flair_text      24574 non-null   object 
 5   removed_by             11564 non-null   object 
 6   total_awards_received  44007 non-null   float64
 7   awarders               33332 non-null   object 
 8   created_utc            183891 non-null  int64  
 9   full_link              183891 non-null  object 
 10  num_comments           183891 non-null  int64  
 11  over_18                183891 non-null  bool   
 12  test                   183891 non-null  int64  
dtypes: bool(1), float64(1), int64(4), object(7)
memory usage: 17.0+ MB


In [10]:
%time df['test'] = df[['title', 'author']].apply(lambda x : str(x['title']*2) if len(x['author']) <= 6 else str(x['title']).split(), axis =1)

Wall time: 4.58 s


In [11]:
%time df['test'] = df[['title', 'author']].swifter.apply(lambda x : str(x['title']*2) if len(x['author']) <= 6 else str(x['title']).split(), axis =1)


Wall time: 4.87 s


In [49]:
%time df['score_2'] = df['score'].apply(lambda x: x/2 -1)

Wall time: 42.9 ms


In [50]:
%time df['score_2_swift'] = df['score'].swifter.apply(lambda x: x/2 - 1)

Wall time: 3.99 ms


In [19]:
df['test'] = df[['score', 'title']].swifter.apply(lambda x: len(str(x['title'])) if x['score'] <2 else len(str(x['title']*2)) , axis =1)

In [20]:
df['test'] = df[['score', 'title']].apply(lambda x: len(str(x['title'])) if x['score'] <2 else len(str(x['title']*2)) , axis =1)

In [22]:
df['test'] = df[['score', 'title']].swifter.apply(lambda x: np.where(x['score'] <2, len(str(x['title'])) , len(str(x['title']*2))) , axis =1)

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183891 entries, 0 to 183890
Data columns (total 13 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   id                     183891 non-null  object 
 1   title                  183890 non-null  object 
 2   score                  183891 non-null  int64  
 3   author                 183891 non-null  object 
 4   author_flair_text      24574 non-null   object 
 5   removed_by             11564 non-null   object 
 6   total_awards_received  44007 non-null   float64
 7   awarders               33332 non-null   object 
 8   created_utc            183891 non-null  int64  
 9   full_link              183891 non-null  object 
 10  num_comments           183891 non-null  int64  
 11  over_18                183891 non-null  bool   
 12  test                   183891 non-null  int64  
dtypes: bool(1), float64(1), int64(4), object(7)
memory usage: 17.0+ MB


In [39]:
def scoring_comment(x):
    if x['num_comments'] == 0:
        return x['score'] *2
    else:
        return x['score']

In [48]:
%time df['test'] = df[['score', 'num_comments']].apply(scoring_comment, axis =1)

Wall time: 3.96 s


In [51]:
%time df['test'] = df[['score', 'num_comments']].swifter.apply(scoring_comment, axis =1)


Wall time: 6.38 s


In [43]:
def scoring_comment_vectorized(x):
    return np.where(x['num_comments'] ==0, x['score']*2, x['score'])

In [52]:
%time df['test'] = df[['score', 'num_comments']].swifter.apply(scoring_comment_vectorized, axis =1)

Wall time: 11 ms


In [53]:
%time df['test'] = df[['score', 'num_comments']].apply(scoring_comment_vectorized, axis =1)

Wall time: 6.25 s
